## Monte Carlo (with Importance Sampling) Pricing of FX Forward CVA under Vasicek Model

Here we use unilateral CVA model: 

$$
{CVA}_{t} = \mathbb{E}_t [ \mathbb{1} \{\tau \leq T  \}  L_{\tau} D(t,T) ]
$$

where $L_{t}$ is the loss, given default rate $R$ and a derivative contract with value $X_t$ at time $t$.  $D(t,T)$ is the time discount factor. 

$$
L_{t}= (1-R) \cdot max(X_t,0)
$$

A model for counterparty default is the stochastic intensity model.  Given a constant default intensity $\lambda^{\tau}$, the probability of default before time $t$ is

$$
P(\tau \leq t) = 1- e^{-\lambda^{\tau} \cdot t}
$$



#### To Simulate the default times by Monte Carlo with Importance Sampling:

The idea is to exclude the default times after expiration. Thus, by inverse transform sampling, 
$$
\tau^i =  -\frac{log(1-u)}{\lambda^{\tau}} 
$$

where $u$ follows uniform distribution of $(0,P(\tau \leq T))$.

Then the new $CVA$ formula becomes:

$$
CVA_t = (\frac{1}{n} \sum_{i}^{n} L_{\tau^{i}}^{i}  D_{t,\tau^{i}}) \cdot P(\tau \leq T)
$$

#### Since we want to price an FX Forward CVA, let the FX Forward be priced in the following way:
$$X_t= X_0 \cdot \frac{Bond^{foreign}}{Bond^{domestic}}$$
where the bond price follows Vasicek model.

In [1]:
import numpy as np
import statistics
import pandas as pd
import time

In [2]:
# initialize values
class Dynamics:
    pass
domestics_bond = Dynamics()
domestics_bond.r0 = 0.04
domestics_bond.theta = 3
domestics_bond.mu = 0.05
domestics_bond.sigma = 0.08

foreign_bond = Dynamics()
foreign_bond.r0 = 0.10
foreign_bond.theta = 4
foreign_bond.mu = 0.08
foreign_bond.sigma = 0.12

class Player:
    pass

counter_party = Player()
counter_party.def_intensity = 0.05 # 0.05?
counter_party.recovery_rate = 0.3

class Contract:
    pass

contract = Contract()
contract.T = 1 

class MC:
    pass

MonteCarlo=MC()
MonteCarlo.N = 500   # Number of timesteps on each path
MonteCarlo.M = 500  # Number of paths.  Change this if necessary.
MonteCarlo.seed = 0

In [3]:
# To calculate probability of default given default intensity lambda, before expiration:
def default_probability(Player,contract):
    
    return 1- np.exp(-Player.def_intensity*contract.T)

In [4]:
# To simulate m default times

def default_time_importance_sampling(MC, player, contract):
    lmbd=player.def_intensity 
    tau=np.zeros(MC.M)
    P=default_probability(player,contract)
    for i in range(MC.M):
        u = np.random.uniform(0, P)
        tau[i]= -(1/lmbd)* np.log(1-u)
    return tau

In [5]:
def vasicek(dynamics,contract, N=1000, seed=777, simulations=10000):    
    #np.random.seed(seed)
    r0,theta,mu,sigma,T=dynamics.r0,dynamics.theta,dynamics.mu,dynamics.sigma, contract.T
    dt = T/float(N)
    mean_vasicek=lambda t:np.exp(-theta*t)*(r0+mu*(np.exp(theta*t)-1))
    var_vasicek=lambda t:((1-np.exp(-2*theta*t))/2*theta)*sigma**2
    normal_matrix=np.random.standard_normal(size=(simulations,N+1))
    rates_paths=np.add(np.array([mean_vasicek(dt*t_) for t_ in range(N+1)]),np.sqrt([var_vasicek(dt*t_) for t_ in range(N+1)])*normal_matrix)
        
    return [x*dt for x in range(N+1)], rates_paths

In [6]:
# def pricer_bond_vasicek_MonteCarlo(dynamics,contract,N=1000, seed=777, simulations=10000):
#     t,rates=vasicek(dynamics,contract,N=N, seed=seed, simulations=simulations)
#     num_paths=len(rates)
#     dt=t[1]
#     exp_integrals=[np.exp(-(np.array(rates[i])*dt).sum()) for i in range(num_paths)]
        
#     return np.mean(exp_integrals)

In [7]:
def MC_Uni_CVA(X0,dynamics_domestic,dynamics_foreign, contract, MC, player):
    R= player.recovery_rate
    P=default_probability(player,contract)
    tau=default_time_importance_sampling(MC, player, contract) # generate M tau's.
    
    t, rate_foreign=vasicek(dynamics_foreign,contract,N=MC.N, seed=777, simulations=MC.M)
    t, rate_domestic=vasicek(dynamics_domestic,contract,N=MC.N, seed=777, simulations=MC.M)
    
    L=np.zeros(MC.M)
    V=np.zeros(MC.M)
    for i in range(MC.M):
        
        
        #bond_foreign= pricer_bond_vasicek_MonteCarlo(dynamics_foreign,contract,N=1000, seed=777, simulations=100000)
        #bond_domestic= pricer_bond_vasicek_MonteCarlo(dynamics_domestic,contract,N=1000, seed=777, simulations=100000)
        #V[i]=bond_foreign[-1]*np.exp(-rate_foreign[-1]*tau[i] )/(bond_domestic[-1]*np.exp(-rate_domestic[-1]*tau[i] ))
        
        index_f=np.argwhere(abs(t-tau[i])< 0.1)[0][0]
        index_d=np.argwhere(abs(t-tau[i])< 0.1)[0][0]
        default_index= index_f if index_f< index_d else index_d
        
        ##num_paths=len(t) - default_index
        #print(default_index)
        #print(num_paths)
        
        rate_f= np.mean(rate_foreign,axis=0)[default_index]
        rate_d= np.mean(rate_domestic,axis=0)[default_index]
        #rate_d= np.mean(rate_domestic[j][default_index])
        dt=t[1]
        exp_integrals_f=[np.exp(-(np.array(rate_foreign[i][default_index:-1])*dt).sum()) for i in range(MC.M)]
        #print(len(exp_integrals_f))
        bond_foreign=np.mean(exp_integrals_f) # bond value at time of default
        exp_integrals_d=[np.exp(-(np.array(rate_domestic[i][default_index:-1])*dt).sum()) for i in range(MC.M)]
        #print(len(exp_integrals_d))
        bond_domestic=np.mean(exp_integrals_d) # bond value at time of default
        V[i]=X0* bond_foreign*np.exp(-rate_f*tau[i] )/(bond_domestic*np.exp(-rate_d*tau[i] )) # discounted price of bond expires at T
        L[i]=(1-R)*np.max( V[i],0)
        
    CVA_0= np.mean(L[i])* P
    std_err= statistics.stdev(L)/np.sqrt(MC.M)
    return CVA_0, std_err
    
    

In [8]:
CVA,std_err=MC_Uni_CVA(X0=1.3, dynamics_domestic=domestics_bond,dynamics_foreign=foreign_bond, contract=contract, MC=MonteCarlo, player=counter_party)

In [9]:
print("CVA of this FX forward is %.4f" %CVA)
print("The standard error of Monte Carlo estimation is %.7f" %std_err)

CVA of this FX forward is 0.0432
The standard error of Monte Carlo estimation is 0.0002380


In [10]:
N=[100,500,1000,5000,10000]
M=[100,500,1000,5000,10000]
df=pd.DataFrame()
for n in N:
    for m in M:
        MonteCarlo.N=n
        MonteCarlo.M=m
        start= time.process_time()
        CVA,std_err=MC_Uni_CVA(X0=1.3, dynamics_domestic=domestics_bond,dynamics_foreign=foreign_bond, contract=contract, MC=MonteCarlo, player=counter_party)
        end=time.process_time()
        timeval=end-start
        cell="{:.2f}s/{:.4f}%".format(timeval, std_err*100)
        df.loc[n,m]=cell


In [11]:
df

,100,500,1000,5000,10000
100,0.19s/0.0975%,3.81s/0.0248%,15.88s/0.0130%,370.34s/0.0041%,1397.20s/0.0028%
500,0.19s/0.0958%,4.08s/0.0230%,15.95s/0.0124%,460.84s/0.0043%,1824.78s/0.0028%
1000,0.30s/0.0970%,5.47s/0.0243%,22.44s/0.0123%,512.66s/0.0041%,1924.73s/0.0028%
5000,0.45s/0.1023%,8.45s/0.0250%,33.45s/0.0130%,824.30s/0.0042%,3297.31s/0.0028%
10000,0.73s/0.1037%,13.55s/0.0239%,52.91s/0.0117%,1282.28s/0.0042%,5124.67s/0.0028%
